In [1]:
import pandas as pd
import numpy as np
import json
import matplotlib.pyplot as plt
import seaborn as sns
import cv2
import os
import math

import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
import torch.optim.lr_scheduler as lr_scheduler
import torchvision 
import torchvision.transforms as transforms 
import torchvision.models as models
from torch.utils.data import Dataset, DataLoader, Subset
from torch.utils.data.sampler import WeightedRandomSampler

from pytorch_lightning import LightningDataModule
from pytorch_lightning import LightningModule
from pytorch_lightning.callbacks import ModelCheckpoint, TQDMProgressBar
from pytorch_lightning.strategies import ParallelStrategy
from pytorch_lightning.utilities.cli import LightningCLI
from pytorch_lightning import Trainer
# from pytorch_lightning import loggers as pl_loggers
from pytorch_lightning.loggers import TensorBoardLogger, WandbLogger
from pytorch_lightning.callbacks import ModelCheckpoint, EarlyStopping, LearningRateMonitor
from pytorch_lightning.plugins import DDPPlugin
from torchmetrics import Accuracy, F1Score, Specificity

import json
import matplotlib.image as image 
import albumentations as A
import albumentations.pytorch
from sklearn.model_selection import train_test_split

import sys
sys.path.append('../')

from utils.dataset_multi import *



/home/beomgon/anaconda3/envs/pl/lib/python3.8/site-packages/tqdm/auto.py:22: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
df = pd.read_csv('../../dataset/df.csv')

In [3]:
df.head()

,image_id,image_dir,category,genus,institutions
0,00000__001,dataset/train_images/000/00/00000__001.jpg,0,Abies,0
1,00000__002,dataset/train_images/000/00/00000__002.jpg,0,Abies,0
2,00000__003,dataset/train_images/000/00/00000__003.jpg,0,Abies,0
3,00000__004,dataset/train_images/000/00/00000__004.jpg,0,Abies,0
4,00000__005,dataset/train_images/000/00/00000__005.jpg,0,Abies,0


In [4]:
df.shape

(839772, 5)

In [5]:
print(len(df.category.unique()))
print(len(df.genus.unique()))
print(len(df.institutions.unique()))

15501
2564
60


In [6]:
df.reset_index(inplace=True, drop=True)
df.head()

,image_id,image_dir,category,genus,institutions
0,00000__001,dataset/train_images/000/00/00000__001.jpg,0,Abies,0
1,00000__002,dataset/train_images/000/00/00000__002.jpg,0,Abies,0
2,00000__003,dataset/train_images/000/00/00000__003.jpg,0,Abies,0
3,00000__004,dataset/train_images/000/00/00000__004.jpg,0,Abies,0
4,00000__005,dataset/train_images/000/00/00000__005.jpg,0,Abies,0


In [7]:
max(df.category.unique())

15504

In [8]:
import custom_models
model = custom_models.__dict__['resnet18'](pretrained=False)

In [9]:
# model

In [10]:
from pl_models_multi import HerbClsModel
num_classes1 = len(df.category.unique())
num_classes2 = len(df.genus.unique())
num_classes3 = len(df.institutions.unique())
model = HerbClsModel(pretrained=False)

=> creating model 'resnet18'


In [11]:
# model

In [12]:
from utils.dataset_utils import LabelEncoder
le = LabelEncoder(df)
df = le.get_integer_labels()

batch_size = 2
dm = HerbDataModule(df, batch_size, transform=train_transforms)


(839772, 5)


/home/beomgon/anaconda3/envs/pl/lib/python3.8/site-packages/pytorch_lightning/core/datamodule.py:95: LightningDeprecationWarning: DataModule property `train_transforms` was deprecated in v1.5 and will be removed in v1.7.
  rank_zero_deprecation(
/home/beomgon/anaconda3/envs/pl/lib/python3.8/site-packages/pytorch_lightning/core/datamodule.py:133: LightningDeprecationWarning: DataModule property `test_transforms` was deprecated in v1.5 and will be removed in v1.7.
  rank_zero_deprecation(


In [13]:
num_classes = len(df.category.unique())

devices = torch.cuda.device_count()
trainer = Trainer(
    max_epochs=3,
    progress_bar_refresh_rate=20,
    # gpus=devices,
    gpus=1,
    # strategy = "ddp",
)
trainer.fit(model, dm)

/home/beomgon/anaconda3/envs/pl/lib/python3.8/site-packages/pytorch_lightning/trainer/connectors/callback_connector.py:96: LightningDeprecationWarning: Setting `Trainer(progress_bar_refresh_rate=20)` is deprecated in v1.5 and will be removed in v1.7. Please pass `pytorch_lightning.callbacks.progress.TQDMProgressBar` with `refresh_rate` directly to the Trainer's `callbacks` argument instead. Or, to disable the progress bar pass `enable_progress_bar = False` to the Trainer.
  rank_zero_deprecation(
GPU available: True, used: True
TPU available: False, using: 0 TPU cores
IPU available: False, using: 0 IPUs
HPU available: False, using: 0 HPUs
/home/beomgon/anaconda3/envs/pl/lib/python3.8/site-packages/pytorch_lightning/core/datamodule.py:88: LightningDeprecationWarning: DataModule property `train_transforms` was deprecated in v1.5 and will be removed in v1.7.
  rank_zero_deprecation(
/home/beomgon/anaconda3/envs/pl/lib/python3.8/site-packages/pytorch_lightning/core/datamodule.py:126: Light

(629829, 5)
(209943, 5)


LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0,1]

  | Name        | Type             | Params
-------------------------------------------------
0 | model       | ResNet           | 11.7 M
1 | fc1         | Linear           | 8.0 M 
2 | fc2         | Linear           | 1.3 M 
3 | fc3         | Linear           | 30.8 K
4 | criterion   | CrossEntropyLoss | 0     
5 | train_acc1  | Accuracy         | 0     
6 | eval_acc1   | Accuracy         | 0     
7 | f1          | F1Score          | 0     
8 | specificity | Specificity      | 0     
-------------------------------------------------
21.0 M    Trainable params
0         Non-trainable params
21.0 M    Total params
83.951    Total estimated model params size (MB)


Epoch 0:   0%|          | 460/419887 [00:14<3:45:16, 31.03it/s, loss=17.2, v_num=14, train_acc=0.000]

/home/beomgon/anaconda3/envs/pl/lib/python3.8/site-packages/pytorch_lightning/trainer/trainer.py:724: UserWarning: Detected KeyboardInterrupt, attempting graceful shutdown...
  rank_zero_warn("Detected KeyboardInterrupt, attempting graceful shutdown...")
